# Ensemble Models

We performed ensembling with soft voting of seven models to significantly improve our highest test accuracy.

[sklearn.ensemble.VotingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html)

## Import Dependencies 

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.python.lib.io import file_io

%matplotlib inline

import keras
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
from sklearn.metrics import *
from keras.engine import Model
from keras.layers import Input, Flatten, Dense, Activation, Conv2D, MaxPool2D, BatchNormalization, Dropout, MaxPooling2D

import skimage
from skimage.transform import rescale, resize

## Import datasets

In [14]:
Resize_pixelsize = 197

In [ ]:
# Function that reads the data from the csv file, increases the size of the images and returns the images and their labels
def get_data(dataset_patj):
    
    file_stream = file_io.FileIO(dataset_path, mode='r')
    data = pd.read_csv(file_stream)
    data[' pixels'] = data[' pixels'].apply(lambda x: [int(pixel) for pixel in x.split()])
    X, Y = data[' pixels'].tolist(), data['emotion'].values
    X = np.array(X, dtype='float32').reshape(-1,48,48,1)
    X = X/255.0
   
    X_res = np.zeros((X.shape[0], Resize_pixelsize,Resize_pixelsize,3))
    for ind in range(X.shape[0]): 
        sample = X[ind]
        sample = sample.reshape(48, 48)
        image_resized = resize(sample, (Resize_pixelsize, Resize_pixelsize), anti_aliasing=True)
        X_res[ind,:,:,:] = image_resized.reshape(Resize_pixelsize,Resize_pixelsize,1)

    Y_res = np.zeros((Y.size, 7))
    Y_res[np.arange(Y.size),Y] = 1    
    
    return  X, X_res, Y_res

dev_dataset_dir = '/content/drive/My Drive/cs230 project/collab/fer2013/dev.csv'
test_dataset_dir = '/content/drive/My Drive/cs230 project/collab/fer2013/test.csv'

X_dev, X_res_dev, Y_dev   = get_data(dev_dataset_dir)
X_test, X_res_test, Y_test   = get_data(test_dataset_dir)

## Import models

In [ ]:
model1 = load_model('/content/drive/My Drive/cs230 project/models/soa-SGD_LR_0.01000-EPOCHS_300-BS_128-DROPOUT_0.3test_acc_0.663.h5')
model2 = load_model('/content/drive/My Drive/cs230 project/models/soa-SGD_LR_0.01000-EPOCHS_300-BS_128-DROPOUT_0.4test_acc_0.657.h5')

Resnet_model = load_model('/content/drive/My Drive/cs230 project/models/tl/ResNet-BEST-73.2.h5')
Resnet_model_wcw = load_model('/content/drive/My Drive/cs230 project/models/tl/ResNet-BEST-WCW-0.677.h5')
Senet_model_wcw = load_model('/content/drive/My Drive/cs230 project/models/tl/SeNet50-WCW-BEST-68.9.h5')
Resnet_Aux_model = load_model("/content/drive/My Drive/cs230 project/models/final/RESNET50-AUX-BEST-72.7.h5")
Resnet_Aux_model_wcw = load_model("/content/drive/My Drive/cs230 project/models/final/RESNET50-WCW-AUX-BEST-72.4.h5")
Senet_Aux_model = load_model('/content/drive/My Drive/cs230 project/models/final/SENET50-AUX-BEST-72.5.h5')
Senet_Aux_model_wcw = load_model('/content/drive/My Drive/cs230 project/models/final/SENET50-WCW-AUX-BEST-71.6.h5')
VGG_Aux_model = load_model("/content/drive/My Drive/cs230 project/models/final/VGG16-AUX-BEST-70.2.h5")

In [ ]:
models_SOA = [model1, model2]
models_TL = [Resnet_model, Resnet_Aux_model_wcw, Senet_Aux_model, Senet_Aux_model_wcw, VGG_Aux_model]
models = models_SOA + models_TL

In [ ]:
for model in models:
    print('\n Model',model)
    print('\n Evaluate on dev data')
    results_dev = model.evaluate(X_res_dev,Y_dev)
    print('dev loss, dev acc:', results_dev)

    print('\n Evaluate on test data')
    results_test = model.evaluate(X_res_test,Y_test)
    print('test loss, test acc:', results_test)

## Ensemble models

In [ ]:
# make an ensemble prediction for multi-class classification
def ensemble_predictions(models_SOA, testX, models_TL, testresX):
  # make predictions
  yhats = np.zeros((len(models_SOA)+len(models_TL),testX.shape[0],7))

  for model_ind in range(len(models_SOA)):
    yhat = models_SOA[model_ind].predict(testX)
    yhats[model_ind,:,:] = yhat

  for model_ind in range(len(models_TL)):
    yhat = models_TL[model_ind].predict(testresX)
    yhats[len(models_SOA)+model_ind,:,:] = yhat

  summed = np.sum(yhats, axis=0)
  result = np.argmax(summed, axis=1)
  return result
 
# evaluate a specific number of members in an ensemble
def evaluate_n_members(models_SOA, testX, models_TL, testresX, testy):
    # select a subset of members
    #subset = members[:n_members]
    #print(len(subset))
    # make prediction
    yhat = ensemble_predictions(models_SOA, testX, models_TL, testresX)
    # calculate accuracy
    return accuracy_score(testy, yhat)

In [ ]:
ens_acc = evaluate_n_members(models_SOA, X_test, models_TL, X_res_test, np.argmax(Y_test, axis=1))
print(ens_acc)